# <center> <h3> <span>Diabetes Disease Classification </span> </h3> </center>

## <span>⬇️ Import Libraries</span>

In [12]:
import pandas as pd
import numpy as np
import time

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [14]:
import warnings
warnings.filterwarnings('ignore')

## <span>📚 Read the Dataset</span>

In [4]:
df = pd.read_csv('diabetes-dataset.csv')
x = df.drop('Outcome',axis=1)
y = df['Outcome']

## <span>✂ Split the Data into Train and Test </span>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

x = X_train.values.T
y = y_train.values

### Hyperparameters

In [15]:
np.random.seed(420)
no_hidden_layers = 2
learning_rate = 0.01   
no_epochs = 100          
number_of_inputs = 8     
output_neurons = 1       

max_no_hidden_layers = 5 

### Creating a dictionary with hyperparameters

In [19]:
def init_hyperparams_dict(max_no_hidden_layers):
    keys = ['no_hidden_layers']    
    for i in range(max_no_hidden_layers):    
        keys.append('no_neurons_hidden_layer_'+str(i))
        keys.append('activation_function_'+str(i))

    keys.extend(['output_activation_function', 'learning_rate', 'no_epochs'])
    hyperparams = dict.fromkeys(keys)    
    return hyperparams


def add_params_dict(hyperparams, no_hidden_layers, max_no_hidden_layers, learning_rate, no_epochs):
    hyperparams['no_hidden_layers'] = no_hidden_layers
    i = 0

    while i < max_no_hidden_layers:
        if i < no_hidden_layers:    
            hyperparams['no_neurons_hidden_layer_'+str(i)] = int(input('Enter number of neurons in Layer'+str(i+1)+': '))
            hyperparams['activation_function_'+str(i)] = str(input('Enter the Activation Function for Layer'+str(i+1)+': '))
        else:                     
            hyperparams["no_neurons_hidden_layer_"+str(i)] = None
            hyperparams["activation_function_"+str(i)] = None
        i += 1
    hyperparams['output_activation_function'] = str(input('Enter the Activation Function for output: '))
    hyperparams['learning_rate'] = learning_rate
    hyperparams['no_epochs'] = no_epochs
    return hyperparams

In [20]:
hyperparams = init_hyperparams_dict(max_no_hidden_layers)
hyperparams = add_params_dict(hyperparams, no_hidden_layers, max_no_hidden_layers, learning_rate, no_epochs)

Enter number of neurons in Layer1: 150
Enter the Activation Function for Layer1: relu
Enter number of neurons in Layer2: 100
Enter the Activation Function for Layer2: relu
Enter the Activation Function for output: sigmoid


In [21]:
hyperparams

{'no_hidden_layers': 2,
 'no_neurons_hidden_layer_0': 150,
 'activation_function_0': 'relu',
 'no_neurons_hidden_layer_1': 100,
 'activation_function_1': 'relu',
 'no_neurons_hidden_layer_2': None,
 'activation_function_2': None,
 'no_neurons_hidden_layer_3': None,
 'activation_function_3': None,
 'no_neurons_hidden_layer_4': None,
 'activation_function_4': None,
 'output_activation_function': 'sigmoid',
 'learning_rate': 0.01,
 'no_epochs': 100}

In [9]:
weights = []
bias = []
def generate_wb(number_of_inputs, hyperparams):
    weights.append(np.random.uniform(low=-0.5, high=0.5, size=(number_of_inputs, hyperparams['no_neurons_hidden_layer_0'])))
    i = 0
    while i<hyperparams['no_hidden_layers']:
        if(i == hyperparams['no_hidden_layers'] - 1): 
    
            weights.append(np.random.uniform(low=-0.5,high=0.5,size=(hyperparams['no_neurons_hidden_layer_'+str(i)], output_neurons)))
            bias.append(np.zeros((hyperparams['no_neurons_hidden_layer_'+str(i)], 1)))
        else:
    
            weights.append(np.random.uniform(low=-0.5,high=0.5,size=(hyperparams['no_neurons_hidden_layer_'+str(i)], hyperparams['no_neurons_hidden_layer_'+str(i+1)])))
            bias.append(np.zeros((hyperparams['no_neurons_hidden_layer_'+str(i)], 1)))
        i+=1
    bias.append(np.zeros((output_neurons, 1)))
    return weights, bias

### Acitvation Functions

In [10]:
def sigmoid(z):
    a=np.zeros([1,1])
    a=1/(1+np.exp(-z))
    return a

def sigmoid_derivative(A):
    return A*(1-A)
    
def ReLU(z):
    return np.maximum(z, 0)

def ReLU_derivative(z):
    return z > 0

def tanh(z):
    z=(np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
    z=1-z**2
    return z

def tanh_derivative(z):
    dz=1-z**2
    return dz

## <span> Forward Propagation</span>

In [11]:
def forward_propagation(weights, bias, hyperparams, x):
    z = []    
    a = []
    i = 0
    while i <= hyperparams['no_hidden_layers']:
        if i == 0:
            z.append(np.matmul(weights[i].T,x) + bias[i])   
        else:
            z.append(np.matmul(weights[i].T, a[i - 1]) + bias[i]) 
        if i == hyperparams['no_hidden_layers']:   
            if(hyperparams['output_activation_function'] == 'sigmoid'):
                a.append(sigmoid(z[i]))
            elif(hyperparams['output_activation_function'] == 'tanh'):
                a.append(tanh(z[i]))
            elif(hyperparams['output_activation_function'] == 'relu'):
                a.append(ReLU(z[i]))
        else:      
            if(hyperparams['activation_function_'+str(i)] == 'sigmoid'):
                a.append(sigmoid(z[i]))
            elif(hyperparams['activation_function_'+str(i)] == 'tanh'):
                a.append(tanh(z[i]))
            elif(hyperparams['activation_function_'+str(i)] == 'relu'):
                a.append(ReLU(z[i]))
        
        i+=1
    
    return z, a

### Log loss

In [12]:
def log_loss_calc(output, y):
    loss = 0
    loss =- 1 * np.sum(y * np.log(output + 1e-10) + ((1 - y) * np.log(1 - output + 1e-10)))
    loss = loss / y.shape[0]
    return loss

### Hinge Loss

In [13]:
def hinge_loss_calc(y, y_hat):
    y_new = np.array([-1 if i==0 else i for i in y])        
    y_hat = np.array([-1 if i==0 else i for i in y_hat])    
    hinge_loss = np.mean([max(0, 1-x*y) for x, y in zip(y_new, y_hat)])
    return hinge_loss


### Backward Propagation

In [14]:
dw = []
db = []
def generate_dwdb(hyperparams):
    dw.clear()
    db.clear()
    for i in range(hyperparams['no_hidden_layers']):
        dw.append(np.zeros_like(weights[i]))
        db.append(np.zeros_like(bias[i]))
    return dw, db

In [15]:
def backward_propagation(a, weights, bias, y, x):
    dz = []
    dz.append((a[no_hidden_layers]) - y)   
    dw.append(np.matmul(a[no_hidden_layers - 1],dz[0].T)/y.shape[0])
    db.append(np.sum(dz[0],axis=0,keepdims=True)/y.shape[0])

    i = no_hidden_layers    
    j = 1    
    while i>0:
    
        if(hyperparams['activation_function_'+str(i-1)] == 'sigmoid'):
            dz.append(np.multiply(np.matmul(weights[i],dz[j-1]),sigmoid_derivative(a[i-1])))
        elif(hyperparams['activation_function_'+str(i-1)] == 'tanh'):
            dz.append(np.multiply(np.matmul(weights[i],dz[j-1]),tanh_derivative(a[i-1])))
        elif(hyperparams['activation_function_'+str(i-1)] == 'relu'):
            dz.append(np.multiply(np.matmul(weights[i],dz[j-1]),ReLU_derivative(a[i-1])))
        if(i!=1):
            dw.append(np.matmul(a[i-2], dz[j].T)/y.shape[0])
            db.append(np.sum(dz[j],axis=0, keepdims=True)/y.shape[0])
        elif(i==1):  
                     
            dw.append(np.matmul(x,dz[j].T)/y.shape[0])
            db.append(np.sum(dz[j], axis=0, keepdims=True)/y.shape[0])
        i-=1
        j+=1
    dz.reverse()
    db.reverse()
    dw.reverse()
    
    i=0
    while i<=no_hidden_layers:
        weights[i] = weights[i]-learning_rate*dw[i]
        bias[i] = bias[i]-learning_rate*db[i]
        i+=1
        
    return weights, bias, dw, db

In [16]:
def accuracy_calc(output):
    y_hat=[]
    correct=0
    accuracy=0
    for i, m in enumerate(output):
        if m<0.5:
            y_hat.append(0)
        elif m>=0.5:
            y_hat.append(1)
            
    correct=np.sum(y==y_hat)
    accuracy=correct/y.shape[0]
    return y_hat, correct, accuracy

In [17]:
t=time.time()    
weights, bias = generate_wb(number_of_inputs, hyperparams)    
for j in range(no_epochs):                                    
    dw, db = generate_dwdb(hyperparams)                       
    z, a = forward_propagation(weights, bias, hyperparams, x) 
    log_loss = log_loss_calc(a[no_hidden_layers][0], y)            
    weights, bias, dw, db = backward_propagation(a, weights, bias, y, x)  
    y_hat, correct, accuracy = accuracy_calc(a[no_hidden_layers][0])

            
    hinge_loss = hinge_loss_calc(y,y_hat)
    print('Total Correct Predictions: ',correct, '/', y.shape[0])
    print('Accuracy for the current Epoch {} = {:.2f}%'.format(j,accuracy*100))

print('*'*50, 'Training Complete', '*'*50)
print('time taken: ' ,time.time()-t)
print('Final LOG cost after training: {:.2f}'.format(log_loss))
print('Final HINGE loss after training : {:.2f}'.format(hinge_loss))
print('Final accuracy after training for {} epochs = {:.2f}%'.format((j + 1),(accuracy * 100)))

Total Correct Predictions:  919 / 1400
Accuracy for the current Epoch 0 = 65.64%
Total Correct Predictions:  481 / 1400
Accuracy for the current Epoch 1 = 34.36%
Total Correct Predictions:  919 / 1400
Accuracy for the current Epoch 2 = 65.64%
Total Correct Predictions:  481 / 1400
Accuracy for the current Epoch 3 = 34.36%
Total Correct Predictions:  919 / 1400
Accuracy for the current Epoch 4 = 65.64%
Total Correct Predictions:  940 / 1400
Accuracy for the current Epoch 5 = 67.14%
Total Correct Predictions:  1056 / 1400
Accuracy for the current Epoch 6 = 75.43%
Total Correct Predictions:  1062 / 1400
Accuracy for the current Epoch 7 = 75.86%
Total Correct Predictions:  709 / 1400
Accuracy for the current Epoch 8 = 50.64%
Total Correct Predictions:  1351 / 1400
Accuracy for the current Epoch 9 = 96.50%
Total Correct Predictions:  1351 / 1400
Accuracy for the current Epoch 10 = 96.50%
Total Correct Predictions:  1361 / 1400
Accuracy for the current Epoch 11 = 97.21%
Total Correct Predict

Total Correct Predictions:  1387 / 1400
Accuracy for the current Epoch 99 = 99.07%
************************************************** Training Complete **************************************************
time taken:  3.016026258468628
Final LOG cost after training: 0.67
Final HINGE loss after training : 0.02
Final accuracy after training for 100 epochs = 99.07%


In [18]:
def predict_output(input_data, weights, bias, hyperparams):
    x = np.array(input_data).reshape(-1, 1) 
    z, a = forward_propagation(weights, bias, hyperparams, x)
    predicted_output = np.max(a[hyperparams['no_hidden_layers']])
    return 1 if predicted_output >= 0.5 else 0

input_data = [1, 85, 66, 29, 0, 26.6, 0.351, 31]
prediction = predict_output(input_data, weights, bias, hyperparams)
print("Prediction:", prediction)

Prediction: 1


In [6]:
import joblib

In [22]:
joblib.dump((weights, bias,hyperparams), 'hyperparameters.joblib')

['hyperparameters.joblib']